In [8]:
import os
import sys

# Check current directory
print("Current working directory:", os.getcwd())
print("\nPython executable:", sys.executable)

# Try to find the file
possible_paths = [
    r'C:\FYP\omniParser\OmniParser',
    '/mnt/c/FYP/omniParser/OmniParser',  # WSL path
    os.getcwd(),  # Current directory
    os.path.dirname(os.path.abspath('')),  # Notebook directory
]

print("\nSearching for streamlit_yolo_ocr.py...")
for path in possible_paths:
    if os.path.exists(path):
        file_path = os.path.join(path, 'streamlit_yolo_ocr.py')
        if os.path.exists(file_path):
            print(f"✅ Found at: {file_path}")
            os.chdir(path)
            print(f"Changed to: {path}")
            break
    else:
        print(f"❌ Path doesn't exist: {path}")

# Also check current directory
if os.path.exists('streamlit_yolo_ocr.py'):
    print("\n✅ File found in current directory!")
    print("Current directory:", os.getcwd())

Current working directory: /content

Python executable: /usr/bin/python3

Searching for streamlit_yolo_ocr.py...
❌ Path doesn't exist: C:\FYP\omniParser\OmniParser
❌ Path doesn't exist: /mnt/c/FYP/omniParser/OmniParser


In [6]:
!streamlit run "C:\FYP\omniParser\OmniParser\streamlit_yolo_ocr.py" --server.port 8501 --server.address 0.0.0.0

Usage: streamlit run [OPTIONS] [TARGET] [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: C:\FYP\omniParser\OmniParser\streamlit_yolo_ocr.py


In [2]:
import torch
cuda_available = torch.cuda.is_available()
cuda_available

True

In [3]:
%pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 118.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 92.7 MB/s eta 0:00:00:00:0100:01


In [2]:
# Check GPU using CUDA environment variables and pynvml (if available)
print("=== GPU Check with Environment Variables ===")

import os

# Check CUDA-related environment variables
cuda_vars = {k: v for k, v in os.environ.items() if 'CUDA' in k.upper() or 'GPU' in k.upper()}
if cuda_vars:
    print("\nCUDA/GPU Environment Variables:")
    for key, value in cuda_vars.items():
        print(f"  {key}: {value}")
else:
    print("\nNo CUDA/GPU environment variables found")

# Try pynvml (NVIDIA Management Library)
try:
    import pynvml
    pynvml.nvmlInit()
    device_count = pynvml.nvmlDeviceGetCount()
    print(f"\n✅ pynvml detected {device_count} GPU(s)")
    
    for i in range(device_count):
        handle = pynvml.nvmlDeviceGetHandleByIndex(i)
        name = pynvml.nvmlDeviceGetName(handle).decode('utf-8')
        mem_info = pynvml.nvmlDeviceGetMemoryInfo(handle)
        print(f"\nGPU {i}:")
        print(f"  Name: {name}")
        print(f"  Memory Total: {mem_info.total / 1024**3:.2f} GB")
        print(f"  Memory Free: {mem_info.free / 1024**3:.2f} GB")
        print(f"  Memory Used: {mem_info.used / 1024**3:.2f} GB")
        
        if 'T4' in name.upper():
            print(f"  🎉 This is a T4 GPU!")
            
except ImportError:
    print("\n⚠️  pynvml not installed (optional library)")
except Exception as e:
    print(f"\n❌ Error with pynvml: {e}")


=== GPU Check with Environment Variables ===

CUDA/GPU Environment Variables:
  NVIDIA_REQUIRE_CUDA: cuda>=12.5 brand=unknown,driver>=470,driver<471 brand=grid,driver>=470,driver<471 brand=tesla,driver>=470,driver<471 brand=nvidia,driver>=470,driver<471 brand=quadro,driver>=470,driver<471 brand=quadrortx,driver>=470,driver<471 brand=nvidiartx,driver>=470,driver<471 brand=vapps,driver>=470,driver<471 brand=vpc,driver>=470,driver<471 brand=vcs,driver>=470,driver<471 brand=vws,driver>=470,driver<471 brand=cloudgaming,driver>=470,driver<471 brand=unknown,driver>=535,driver<536 brand=grid,driver>=535,driver<536 brand=tesla,driver>=535,driver<536 brand=nvidia,driver>=535,driver<536 brand=quadro,driver>=535,driver<536 brand=quadrortx,driver>=535,driver<536 brand=nvidiartx,driver>=535,driver<536 brand=vapps,driver>=535,driver<536 brand=vpc,driver>=535,driver<536 brand=vcs,driver>=535,driver<536 brand=vws,driver>=535,driver<536 brand=cloudgaming,driver>=535,driver<536 brand=unknown,driver>=550,

In [ ]:
# Final Summary
print("=" * 50)
print("FINAL GPU AVAILABILITY SUMMARY")
print("=" * 50)

# Quick check
has_gpu = False
gpu_name = "Unknown"

# Check TensorFlow
try:
    import tensorflow as tf
    gpus = tf.config.list_physical_devices('GPU')
    if len(gpus) > 0:
        has_gpu = True
        print("✅ GPU detected via TensorFlow")
except:
    pass

# Check nvidia-smi
try:
    result = subprocess.run(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], 
                          capture_output=True, text=True, timeout=5)
    if result.returncode == 0 and result.stdout.strip():
        has_gpu = True
        gpu_name = result.stdout.strip().split('\n')[0]
        print(f"✅ GPU detected via nvidia-smi: {gpu_name}")
        if 'T4' in gpu_name.upper():
            print("🎉 T4 GPU is AVAILABLE!")
        else:
            print(f"⚠️  GPU is available but not T4 (found: {gpu_name})")
except:
    pass

if not has_gpu:
    print("❌ No GPU detected")
    print("💡 Make sure you've selected a GPU runtime in Colab")


In [ ]:
# Check GPU using nvidia-smi (system command)
import subprocess
import sys

print("=== GPU Check with nvidia-smi ===")
try:
    result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total,driver_version', '--format=csv'], 
                          capture_output=True, text=True, timeout=10)
    if result.returncode == 0:
        print("\n✅ nvidia-smi output:")
        print(result.stdout)
        
        # Check if T4 is mentioned
        if 'T4' in result.stdout.upper():
            print("\n🎉 T4 GPU is detected!")
        else:
            print("\n⚠️  T4 GPU not found in nvidia-smi output")
    else:
        print("❌ nvidia-smi command failed")
        print(f"Error: {result.stderr}")
except FileNotFoundError:
    print("❌ nvidia-smi command not found (GPU likely not available)")
except subprocess.TimeoutExpired:
    print("❌ nvidia-smi command timed out")
except Exception as e:
    print(f"❌ Error running nvidia-smi: {e}")
